In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import torch
torch.set_printoptions(precision=10)

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from models.ranked_transformer import HsqcRankedTransformer

from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.oneD_dataset import OneDDataModule

from datasets.optional_2d_folder_dataset import OptionalInputDataModule
import yaml
from pytorch_lightning.loggers import TensorBoardLogger
torch.set_float32_matmul_precision('medium')

In [9]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")


In [2]:
from datasets.dataset_utils import  fp_loader_configer
fp_loader_configer.select_version("MFP_Specific_Radius")
fp_loader = fp_loader_configer.fp_loader


Here we compare models' performance over 7 kinds of inputs


In [5]:
# load model 

checkpoint_path = Path("/root/MorganFP_prediction/reproduce_previous_works/fix_combining_dataset_load_mfp_bug/train_on_all_data_possible/only_c_trial_1/checkpoints/epoch=16-step=27472.ckpt")
model_path = checkpoint_path.parents[1]


hyperpaerameters_path = model_path / "hparams.yaml"




with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
    
FP_building_type = hparams['FP_building_type'].split("_")[-1]
only_2d = not hparams['use_oneD_NMR_no_solvent']
fp_loader.setup(only_2d=only_2d,FP_building_type=FP_building_type)
fp_loader.set_max_radius(int(hparams['FP_choice'].split("_")[-1][1:]), only_2d=only_2d)


del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
# hparams['use_peak_values'] = False




finish entropy list


In [6]:
model = HsqcRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
# model = HsqcRankedTransformer(**hparams)
model.change_ranker_for_testing()
# model.change_ranker_for_testing(test_ranking_set_path = "/workspace/ranking_sets_cleaned_by_inchi/SMILES_R0_to_R4_reduced_FP_ranking_sets_only_all_info_molecules/test/rankingset.pt")
model=model.to("cuda")

Initialized SignCoordinateEncoder[384] with dims [180, 180, 24] and 2 positional encoders. 24 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
None


In [7]:
datamodule = OptionalInputDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], input_src=["HSQC", "oneD_NMR"], batch_size=hparams['bs'], parser_args=hparams)
datamodule.setup("test")
loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR = \
    datamodule.test_dataloader()
    
    
# datamodule = OneDDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], batch_size=hparams['bs'], parser_args=hparams) 
# datamodule.setup("test")
# loader_only_C_NMR = datamodule.test_dataloader()
    


In [8]:
trainer = Trainer()
metric_to_focus = [
    'rank_1',
    'rank_5',
    "mean_rank",
    'cos',
    'f1'    
]
def show_model_performance(model, out_file ="temp_model_show.txt"):
    # loaders = [loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR]
    # names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
    loaders = [ loader_only_C_NMR]

    names = [ "only_C_NMR"]
    # names = ["only_hsqc"]
    with open(out_file, "w") as file:
        for loader, name in zip(loaders, names):
            result =  trainer.test(model, dataloaders=loader) 
            # result = trainer.test(model, datamodule, ckpt_path = checkpoint_path)
            file.write(f"\n{name:10}: ")
            for m in metric_to_focus:
                file.write(f"{m}: {result[0]['test/mean_'+m]:.4f}, ")
    return result

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [10]:
show_model_performance(model);


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

{'on_epoch': True, 'sync_dist': True} 




┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │    0.9945566058158875     │
│   test/mean_active_bits   │     152.267333984375      │
│     test/mean_ce_loss     │    0.02348923496901989    │
│       test/mean_cos       │    0.8844977617263794     │
│       test/mean_f1        │     0.893922746181488     │
│     test/mean_jaccard     │    0.8027695417404175     │
│    test/mean_mean_rank    │     4.443440914154053     │
│    test/mean_neg_loss     │   0.006064091343432665    │
│    test/mean_pos_loss     │   0.017425144091248512    │
│  test/mean_pos_neg_loss   │    0.02348923496901989    │
│    test/mean_precision    │    0.9426888823509216     │
│     test/mean_rank_1      │    0.9444172978401184     │
│     test/mean_rank_10     │    0.9878743290901184     │
│     test/mean_rank_5      │    0.9849446415901184     │
│     test/mean_recall      │    0.8502824306488037     │
└───────────────────────────┴───────────────────────────┘

In [48]:
model.ranker.data

tensor([[0.1054092571, 0.1054092571, 0.1054092571,  ..., 0.0000000000,
         0.0000000000, 0.0000000000],
        [0.0000000000, 0.0000000000, 0.0000000000,  ..., 0.0000000000,
         0.0000000000, 0.0000000000],
        [0.0000000000, 0.0000000000, 0.0000000000,  ..., 0.0000000000,
         0.0000000000, 0.0000000000],
        ...,
        [0.0778498948, 0.0778498948, 0.0778498948,  ..., 0.0000000000,
         0.0000000000, 0.0000000000],
        [0.0655121803, 0.0655121803, 0.0655121803,  ..., 0.0000000000,
         0.0000000000, 0.0000000000],
        [0.0000000000, 0.0000000000, 0.0000000000,  ..., 0.0000000000,
         0.0000000000, 0.0000000000]])

In [8]:
# spectra-flexible model
# usually not gonna use it because it tests right away after training 

# load model 

model_path = Path("/root/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1")
hyperpaerameters_path = model_path / "hparams.yaml"

checkpoint_path = model_path / "checkpoints/epoch=41-step=35994.ckpt"
with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
hparams['use_peak_values'] = False

model = OptionalInputRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
model.change_ranker_for_testing()

flexible_model_result = trainer.test(model, datamodule)


FileNotFoundError: [Errno 2] No such file or directory: '/root/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1/hparams.yaml'

In [ ]:

names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
out_file = "temp_model_show.txt"
with open(out_file, "w") as file:
    for name in  names:
        file.write(f"\n{name:10}: ")
        for m in metric_to_focus:
            file.write(f"{m}: {flexible_model_result[0]['test_mean_'+m+'_'+name]:.4f}, ")


In [ ]:
flexible_model_result[0] .keys()    


dict_keys(['test_mean_ce_loss_all_inputs', 'test_mean_pos_loss_all_inputs', 'test_mean_neg_loss_all_inputs', 'test_mean_pos_neg_loss_all_inputs', 'test_mean_cos_all_inputs', 'test_mean_active_bits_all_inputs', 'test_mean_f1_all_inputs', 'test_mean_precision_all_inputs', 'test_mean_recall_all_inputs', 'test_mean_accuracy_all_inputs', 'test_mean_mean_rank_all_inputs', 'test_mean_rank_1_all_inputs', 'test_mean_rank_5_all_inputs', 'test_mean_rank_10_all_inputs', 'test_mean_ce_loss_HSQC_H_NMR', 'test_mean_pos_loss_HSQC_H_NMR', 'test_mean_neg_loss_HSQC_H_NMR', 'test_mean_pos_neg_loss_HSQC_H_NMR', 'test_mean_cos_HSQC_H_NMR', 'test_mean_active_bits_HSQC_H_NMR', 'test_mean_f1_HSQC_H_NMR', 'test_mean_precision_HSQC_H_NMR', 'test_mean_recall_HSQC_H_NMR', 'test_mean_accuracy_HSQC_H_NMR', 'test_mean_mean_rank_HSQC_H_NMR', 'test_mean_rank_1_HSQC_H_NMR', 'test_mean_rank_5_HSQC_H_NMR', 'test_mean_rank_10_HSQC_H_NMR', 'test_mean_ce_loss_HSQC_C_NMR', 'test_mean_pos_loss_HSQC_C_NMR', 'test_mean_neg_loss_